In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy import stats
import random
%matplotlib inline

dat = pd.read_csv("/content/drive/MyDrive/Dev/vo3_q_data_x.csv")
dat = dat.loc[:,['cm_dummy','sex','age','gamesecond','TVwatch_day']]
dat.head()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,cm_dummy,sex,age,gamesecond,TVwatch_day
0,0,1,44.5,0,33.427600
1,0,1,34.5,0,31.542862
2,0,1,24.5,0,37.825805
3,0,1,44.5,0,36.345911
4,0,1,34.5,0,49.344942


In [2]:
pip install causalml

     |████████████████████████████████| 406 kB 4.3 MB/s 
     |████████████████████████████████| 564 kB 26.8 MB/s 
     |████████████████████████████████| 1.8 MB 2.4 MB/s 
     |████████████████████████████████| 718 kB 44.8 MB/s 
     |████████████████████████████████| 750.6 MB 8.7 kB/s 
  Created wheel for causalml: filename=causalml-0.12.3-cp37-cp37m-linux_x86_64.whl size=1050922 sha256=3499555aff828b638c7b8f0f73cbeae131ad0653785320eda8ff7131a149fb7c
  Stored in directory: /root/.cache/pip/wheels/d7/5e/47/4e965cf8e87c6c10cd7361d910acccd08a1de459f25d55fd23
Successfully built causalml
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have to

In [ ]:
#Propensity Score Estimation
from causalml.propensity import ElasticNetPropensityModel

X=dat.loc[:,['sex','age','gamesecond','TVwatch_day']]
y=dat.loc[:,['cm_dummy']]


pm = ElasticNetPropensityModel(n_fold=5, random_state=42)
ps = pm.fit_predict(X, y)

In [5]:
ps

array([0.49669072, 0.49746874, 0.49829786, ..., 0.34245638, 0.34245919,
       0.34245636])

In [24]:
#Propensity Score Matching
from causalml.match import NearestNeighborMatch, create_table_one

TREATMENT_COL="cm_dummy"
SCORE_COLS=['sex','age','gamesecond','TVwatch_day','PS']
GROUPBY_COL='PS'
X['PS']=ps
X['cm_dummy']=y['cm_dummy']
df = pd.DataFrame()
df = X.copy()

print(df)

psm = NearestNeighborMatch(replace=False,ratio=1,random_state=42)
matched = psm.match(data=df,treatment_col=TREATMENT_COL,score_cols=SCORE_COLS)

#create_table_one(data=matched,
#                 treatment_col=TREATMENT_COL,
#                 features=covariates)

      sex   age  gamesecond  TVwatch_day        PS  cm_dummy
0       1  44.5           0    33.427600  0.496691         0
1       1  34.5           0    31.542862  0.497469         0
2       1  24.5           0    37.825805  0.498298         0
3       1  44.5           0    36.345911  0.496709         0
4       1  34.5           0    49.344942  0.497580         0
...   ...   ...         ...          ...       ...       ...
9995    1  24.5       97226  1172.051927  0.342452         0
9996    1  24.5       97224  1173.538256  0.342464         0
9997    1  24.5       97227  1173.073186  0.342456         0
9998    1  24.5       97225  1173.016481  0.342459         0
9999    1  24.5       97225  1172.514651  0.342456         0

[10000 rows x 6 columns]


TypeError: ignored